https://github.com/explosion/spaCy/blob/master/LICENSE

Example of training an additional entity type

This script shows how to add a new entity type to an existing pre-trained NER
model. To keep the example short and simple, only four sentences are provided
as examples. In practice, you'll need many more — a few hundred would be a
good start. You will also likely need to mix in examples of other entity
types, which might be obtained by running the entity recognizer over unlabelled
sentences, and adding their annotations to the training set.

The actual training is performed by looping over the examples, and calling
`nlp.entity.update()`. The `update()` method steps through the words of the
input. At each word, it makes a prediction. It then consults the annotations
provided on the GoldParse instance, to see whether it was right. If it was
wrong, it adjusts its weights so that the correct action will score higher
next time.

After training your model, you can save it to a directory. We recommend
wrapping models as Python packages, for ease of deployment.

For more details, see the documentation:
* Training: https://spacy.io/usage/training#section-ner
* NER: https://spacy.io/usage/linguistic-features#named-entities

In [1]:
import random
from pathlib import Path
import spacy

In [2]:
# new entity label
LABEL = 'ANIMAL'

In [3]:
# training data
# Note: If you're using an existing model, make sure to mix in examples of
# other entity types that spaCy correctly recognized before. Otherwise, your
# model might learn the new type, but "forget" what it previously knew.
# https://explosion.ai/blog/pseudo-rehearsal-catastrophic-forgetting
TRAIN_DATA = [
    ("Horses are too tall and they pretend to care about your feelings", {
        'entities': [(0, 6, 'ANIMAL')]
    }),

    ("Do they bite?", {
        'entities': []
    }),

    ("horses are too tall and they pretend to care about your feelings", {
        'entities': [(0, 6, 'ANIMAL')]
    }),

    ("horses pretend to care about your feelings", {
        'entities': [(0, 6, 'ANIMAL')]
    }),

    ("they pretend to care about your feelings, those horses", {
        'entities': [(48, 54, 'ANIMAL')]
    }),

    ("horses?", {
        'entities': [(0, 6, 'ANIMAL')]
    })
]

In [4]:
nlp = spacy.blank('en')

In [5]:
ner = nlp.create_pipe('ner')
nlp.add_pipe(ner)

In [6]:
ner.add_label(LABEL)   # add new entity label to entity recognizer

In [7]:
# Note that 'begin_training' initializes the models, so it'll zero out
# existing entity types.
optimizer = nlp.begin_training()

In [8]:
# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    for itn in range(20):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in TRAIN_DATA:
            nlp.update([text], [annotations], sgd=optimizer, drop=0.35,
                       losses=losses)
        print(losses)

{'ner': 18.435354814301867}
{'ner': 8.046508206126969}
{'ner': 6.5207909090344245}
{'ner': 1.9985643577822343}
{'ner': 1.24925943804981}
{'ner': 2.0000000000152793}
{'ner': 2.196974158287136}
{'ner': 1.9964701116472081}
{'ner': 1.1266417879729893}
{'ner': 3.721874032300565e-12}
{'ner': 0.00013615695945970124}
{'ner': 0.0005255360156585591}
{'ner': 3.1043147026457582e-12}
{'ner': 3.242987489675821e-09}
{'ner': 0.09188444533001185}
{'ner': 6.126406952439933e-17}
{'ner': 2.9525606308057107e-16}
{'ner': 2.181247467067094e-19}
{'ner': 6.604785017077264e-24}
{'ner': 1.0096982251651984e-26}


In [9]:
# test the trained model
test_text = 'Do you like horses?'
doc = nlp(test_text)
print("Entities in '%s'" % test_text)
for ent in doc.ents:
    print(ent.label_, ent.text)

Entities in 'Do you like horses?'
